## Data preprocessing

### drop unnescessary columns

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.init as init

import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plts
import seaborn as sns
%matplotlib inline

data = pd.read_csv('/Users/ponynie/Developer/Python_Code/IntroDMLabChula/Final_Project/children anemia.csv')
data.dropna(inplace=True)

data.drop(['Anemia level.1', 'Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)', 'When child put to breast', 'Current marital status'], axis=1, inplace=True)

data = data[data['Had fever in last two weeks'].isin(['Yes', 'No'])]
data = data[data['Taking iron pills, sprinkles or syrup'].isin(['Yes', 'No'])]

data.head(10)


,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Births in last five years,Age of respondent at 1st birth,Anemia level,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Currently residing with husband/partner,Had fever in last two weeks,Hemoglobin level adjusted for altitude (g/dl - 1 decimal),"Taking iron pills, sprinkles or syrup"
3,25-29,Urban,Secondary,Richest,1,25,Moderate,Yes,No,Living with her,No,114.0,No
6,35-39,Urban,Secondary,Richest,2,32,Not anemic,Yes,No,Living with her,No,102.0,Yes
9,20-24,Urban,Secondary,Richest,1,19,Moderate,Yes,No,Living with her,No,113.0,Yes
12,25-29,Urban,Higher,Richest,1,24,Mild,Yes,No,Living with her,No,109.0,No
13,20-24,Urban,Higher,Richest,2,19,Mild,Yes,No,Living with her,No,96.0,Yes
18,25-29,Urban,Secondary,Richest,2,22,Moderate,Yes,No,Living with her,No,96.0,Yes
21,25-29,Urban,Secondary,Richer,1,23,Mild,Yes,No,Living with her,No,96.0,No
22,25-29,Urban,Secondary,Richer,2,20,Mild,Yes,No,Living with her,No,115.0,Yes
25,25-29,Urban,Secondary,Richer,2,20,Not anemic,Yes,No,Staying elsewhere,No,110.0,No
36,30-34,Urban,No education,Richest,3,25,Not anemic,Yes,No,Living with her,No,138.0,No


### Maps categorical to codes

In [2]:
categorical_columns = ['Age in 5-year groups', 'Type of place of residence', 'Highest educational level', 'Wealth index combined', 'Have mosquito bed net for sleeping (from household questionnaire)', 'Smokes cigarettes', 'Currently residing with husband/partner', 'Had fever in last two weeks', 'Taking iron pills, sprinkles or syrup']
numerical_columns = ['Births in last five years', 'Age of respondent at 1st birth', 'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)']
label_columns = ['Anemia level']

for category in categorical_columns:
    data[category] = data[category].astype('category')
data['Anemia level'] = data['Anemia level'].astype('category')

for categorical in categorical_columns:
    print(data[categorical].cat.categories, categorical)
print(data['Anemia level'].cat.categories, "Label")

Index(['15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49'], dtype='object') Age in 5-year groups
Index(['Rural', 'Urban'], dtype='object') Type of place of residence
Index(['Higher', 'No education', 'Primary', 'Secondary'], dtype='object') Highest educational level
Index(['Middle', 'Poorer', 'Poorest', 'Richer', 'Richest'], dtype='object') Wealth index combined
Index(['No', 'Yes'], dtype='object') Have mosquito bed net for sleeping (from household questionnaire)
Index(['No', 'Yes'], dtype='object') Smokes cigarettes
Index(['Living with her', 'Staying elsewhere'], dtype='object') Currently residing with husband/partner
Index(['No', 'Yes'], dtype='object') Had fever in last two weeks
Index(['No', 'Yes'], dtype='object') Taking iron pills, sprinkles or syrup
Index(['Mild', 'Moderate', 'Not anemic', 'Severe'], dtype='object') Label


### Normalized numerical columns

In [3]:

for col in numerical_columns:
    data[col] = (data[col] - data[col].min())/ (data[col].max() - data[col].min())

data[numerical_columns].head(10)

,Births in last five years,Age of respondent at 1st birth,Hemoglobin level adjusted for altitude (g/dl - 1 decimal)
3,0.000000,0.419355,0.602837
6,0.333333,0.645161,0.517730
9,0.000000,0.225806,0.595745
12,0.000000,0.387097,0.567376
13,0.333333,0.225806,0.475177
18,0.333333,0.322581,0.475177
21,0.000000,0.354839,0.475177
22,0.333333,0.258065,0.609929
25,0.333333,0.258065,0.574468
36,0.666667,0.419355,0.773050


In [4]:
for col in categorical_columns:
    data[col] = (data[col].cat.codes.values - data[col].cat.codes.values.min()) / (data[col].cat.codes.values.max() - data[col].cat.codes.values.min())
    
data[categorical_columns].head(10)

,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Currently residing with husband/partner,Had fever in last two weeks,"Taking iron pills, sprinkles or syrup"
3,0.333333,1.0,1.000000,1.00,1.0,0.0,0.0,0.0,0.0
6,0.666667,1.0,1.000000,1.00,1.0,0.0,0.0,0.0,1.0
9,0.166667,1.0,1.000000,1.00,1.0,0.0,0.0,0.0,1.0
12,0.333333,1.0,0.000000,1.00,1.0,0.0,0.0,0.0,0.0
13,0.166667,1.0,0.000000,1.00,1.0,0.0,0.0,0.0,1.0
18,0.333333,1.0,1.000000,1.00,1.0,0.0,0.0,0.0,1.0
21,0.333333,1.0,1.000000,0.75,1.0,0.0,0.0,0.0,0.0
22,0.333333,1.0,1.000000,0.75,1.0,0.0,0.0,0.0,1.0
25,0.333333,1.0,1.000000,0.75,1.0,0.0,1.0,0.0,0.0
36,0.500000,1.0,0.333333,1.00,1.0,0.0,0.0,0.0,0.0


### Create nparray of categorical matrix

In [5]:
categorical_np = [data[i] for i in categorical_columns]
categorical_data = np.stack(categorical_np, 1)
categorical_data[:10]

array([[0.33333333, 1.        , 1.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.66666667, 1.        , 1.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.16666667, 1.        , 1.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.16666667, 1.        , 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 1.        , 1.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 1.        , 1.        , 0.75      , 1.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.33333333, 1.        , 1.        , 0.75      , 1.        ,
        0.        , 0.        , 0.        , 1.        ],


### Convert to Categorical Tensor

In [6]:
categorical_data = torch.tensor(categorical_data, dtype=torch.float64)
categorical_data[:10]

tensor([[0.3333, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6667, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.1667, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.3333, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1667, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.3333, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.3333, 1.0000, 1.0000, 0.7500, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 1.0000, 1.0000, 0.7500, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.3333, 1.0000, 1.0000, 0.7500, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.5000, 1.0000, 0.3333, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

### Create nparray of numerical matrix and convert to Tensor

In [7]:
numerical_data = np.stack([data[i].values for i in numerical_columns], 1)
numerical_data = torch.tensor(numerical_data, dtype=torch.float)
numerical_data[:10]

tensor([[0.0000, 0.4194, 0.6028],
        [0.3333, 0.6452, 0.5177],
        [0.0000, 0.2258, 0.5957],
        [0.0000, 0.3871, 0.5674],
        [0.3333, 0.2258, 0.4752],
        [0.3333, 0.3226, 0.4752],
        [0.0000, 0.3548, 0.4752],
        [0.3333, 0.2581, 0.6099],
        [0.3333, 0.2581, 0.5745],
        [0.6667, 0.4194, 0.7730]])

### Create label's Tensor

In [8]:
outputs = torch.tensor(data['Anemia level'].cat.codes.values).flatten()
outputs[200:250]

tensor([2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2,
        3, 2, 0, 2, 1, 2, 2, 1, 2, 0, 2, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2,
        0, 0], dtype=torch.int8)

### Check correctness of dimension

In [9]:
categorical_data.shape, numerical_data.shape, outputs.shape

(torch.Size([5855, 9]), torch.Size([5855, 3]), torch.Size([5855]))

## Create Model

### Split the data to train and test set

In [10]:
total_records = data.shape[0]
test_records = int(total_records * .2) # 20% of the data for testing
train_records = total_records - test_records # 80% of the data for training

categorical_train_data = categorical_data[:train_records]
categorical_test_data = categorical_data[train_records:]
numerical_train_data = numerical_data[:train_records]
numerical_test_data = numerical_data[train_records:]
train_outputs = outputs[:train_records]
test_outputs = outputs[train_records:]

print(categorical_train_data.shape, categorical_test_data.shape)
print(numerical_train_data.shape, numerical_test_data.shape)
print(train_outputs.shape, test_outputs.shape)



torch.Size([4684, 9]) torch.Size([1171, 9])
torch.Size([4684, 3]) torch.Size([1171, 3])
torch.Size([4684]) torch.Size([1171])


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, numerical_features_size, categorical_features_size, hidden_size, output_size):
        super().__init__()
        all_features_size = numerical_features_size + categorical_features_size
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(all_features_size, hidden_size),
            nn.BatchNorm1d(hidden_size),  
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),  
            nn.Sigmoid(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
train_data = torch.cat((categorical_train_data, numerical_train_data), dim=1).requires_grad_(True)
test_data = torch.cat((categorical_test_data, numerical_test_data), dim=1).requires_grad_(True)
print(train_data)


tensor([[0.3333, 1.0000, 1.0000,  ..., 0.0000, 0.4194, 0.6028],
        [0.6667, 1.0000, 1.0000,  ..., 0.3333, 0.6452, 0.5177],
        [0.1667, 1.0000, 1.0000,  ..., 0.0000, 0.2258, 0.5957],
        ...,
        [0.5000, 0.0000, 0.0000,  ..., 0.3333, 0.4839, 0.5248],
        [0.5000, 0.0000, 1.0000,  ..., 0.0000, 0.3226, 0.2979],
        [0.3333, 0.0000, 1.0000,  ..., 0.3333, 0.2258, 0.5532]],
       dtype=torch.float64, requires_grad=True)


In [13]:
train_data = train_data.float()
test_data = test_data.float()
train_outputs = train_outputs.long()
test_outputs = test_outputs.long()
print(train_outputs)

tensor([1, 2, 1,  ..., 1, 2, 2])


In [14]:
numerical_features_size = numerical_data.shape[1]
categorical_features_size = categorical_data.shape[1]
hidden_size = 40
num_epochs = 20
learning_rate = 0.05
output_size = 4
batch_size = 100

model = NeuralNetwork(numerical_features_size, categorical_features_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model.to(device)
train_data = train_data.to(device)
test_data = test_data.to(device)
train_outputs = train_outputs.to(device)
test_outputs = test_outputs.to(device)

In [16]:
train_dataset = TensorDataset(train_data, train_outputs)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [17]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 10 == 9:  # Print every 100 batches
            print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}")
print("Training complete.")


Epoch 1, Batch 10, Loss: 0.1274
Epoch 1, Batch 20, Loss: 0.1147
Epoch 1, Batch 30, Loss: 0.1154
Epoch 1, Batch 40, Loss: 0.1133
Epoch 1/20, Training Loss: 1.106758713722229
Epoch 2, Batch 10, Loss: 0.1126
Epoch 2, Batch 20, Loss: 0.1123
Epoch 2, Batch 30, Loss: 0.1160
Epoch 2, Batch 40, Loss: 0.1130
Epoch 2/20, Training Loss: 1.198256492614746
Epoch 3, Batch 10, Loss: 0.1137
Epoch 3, Batch 20, Loss: 0.1114
Epoch 3, Batch 30, Loss: 0.1161
Epoch 3, Batch 40, Loss: 0.1108
Epoch 3/20, Training Loss: 1.1034367084503174
Epoch 4, Batch 10, Loss: 0.1122
Epoch 4, Batch 20, Loss: 0.1110
Epoch 4, Batch 30, Loss: 0.1120
Epoch 4, Batch 40, Loss: 0.1147
Epoch 4/20, Training Loss: 1.1428571939468384
Epoch 5, Batch 10, Loss: 0.1113
Epoch 5, Batch 20, Loss: 0.1128
Epoch 5, Batch 30, Loss: 0.1114
Epoch 5, Batch 40, Loss: 0.1133
Epoch 5/20, Training Loss: 1.0946321487426758
Epoch 6, Batch 10, Loss: 0.1127
Epoch 6, Batch 20, Loss: 0.1111
Epoch 6, Batch 30, Loss: 0.1147
Epoch 6, Batch 40, Loss: 0.1129
Epoc

In [18]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(test_data)
    _, predicted = torch.max(outputs.data, 1)
    print(predicted)
    print(test_outputs)
    total += test_outputs.size(0)
    correct += (predicted == test_outputs).sum().item()
    print(correct)
    print(total)

print(f"Accuracy of the model on the test data: {(100 * correct / total)}%")

tensor([2, 2, 2,  ..., 2, 2, 1])
tensor([1, 1, 2,  ..., 2, 2, 2])
502
1171
Accuracy of the model on the test data: 42.86934244235696%
